In [2]:
# Using the company facts API, we can get a ton of data about many companies
# But tags aren't always the same across companies, and can vary a lot
# Let's analyze this

In [22]:
import scraper
import pandas as pd

In [4]:
companies = scraper.get_companies()

In [5]:
facts = companies[0].get_facts() # example sets of facts for the first company

In [ ]:
# facts["facts"] has keys representing different taxonomies
# e.g.: dei, us-gaap, invest and srt. 

# facts["facts"]["<taxonomy>"]["<tag name>"]["units"] contains keys for each kind of unit
# This is commonly just 'USD'

# facts["facts"]["taxonomy"]["<tag name>"]["units"]["<unit name>"] contains a list of the tag details
# Each is a dict with end, val, accn, fy (fiscal year), fp (quarter), form (e.g. 10-Q), filed (the date)
    # end is "conformed period of report", i.e. the end date of the period being reported on (more important)
    # filed is when the filing was filed (duh)

# Process the data (e.g. into a dataframe)
    # Remove duplicate logs of the same value, only log when it changed and merge dates
# Then compare across companies (what tags are shared across companies a lot?)

In [ ]:
# There is a dataframe for the tags and their descriptions
# Then there is a dataframe for each tag, containing the data
# There is a column for each unit (usually only 1, but there can be more)

tag_info = []
tags_data = {} # tags_data[tag][unit] gives a df for that unit

for gaap_tag in facts["facts"]["us-gaap"]:
    tag_info.append([gaap_tag, facts["facts"]["us-gaap"][gaap_tag]["description"]])

    tags_data[gaap_tag] = {}
    for unit in facts["facts"]["us-gaap"][gaap_tag]["units"].keys():

        # Needs rewriting / cleaning up
        consistent_keys = set() # for some reason, the keys aren't always consistent
        for rec in facts["facts"]["us-gaap"][gaap_tag]["units"][unit]:
            if len(consistent_keys) == 0:
                for key in rec.keys():
                    consistent_keys.add(key)
            else:
                new_keys = set()
                for key in consistent_keys:
                    if key in rec.keys():
                        new_keys.add(key)
                consistent_keys = new_keys

        data = []
        for rec in facts["facts"]["us-gaap"][gaap_tag]["units"][unit]:
            data.append([rec[key] for key in consistent_keys])

        tags_data[gaap_tag][unit] = pd.DataFrame(data, columns=list(consistent_keys))    
tags_df = pd.DataFrame(tag_info, columns=["name", "description"])    

In [ ]:
# TODO:
    # Go through top ~500 companies.
    # For each, check the us gaap tag units
    # Check whenever there's multiple (e.g. 'pure' or 'USD/share')
    # Is it a good idea to pick one unit over another, or could both be useful?